<div><img src = "https://jacademy.juventus.com/media/images/juventus-academy-toronto/Home%20page%20banner.jpg" 
        width = 500 alt="Toronto Skyline"> </div>
<h1 align=center><font size = 5>Segmenting & Clustering Neighborhoods in Toronto</font></h1>

This notebook is the 1st part of the ***Capstone Project in Applied Data Science for Coursera & IBM***
The aim is to obtain information about the neighborhoods in **Toronto, Canada**, so i can use this information to further analyse the venues available.
## 1. Scraping list of postal codes from Toronto
I'll start this project by scrapping information from the following url (wikipedia):

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Firstly and foremost I need the following libraries:

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup as bs
import requests
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [2]:
path = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Now I scrape the wikipedia url for the full information on the presented table, using beautiful soup:

In [3]:
#Request to website
req_parsed = bs(requests.get(path).text, 'html.parser')
#print(req_parsed)
#Find table
table = req_parsed.find('table',{'class':'wikitable sortable'})
content = []
columns = ['PostalCode', 'Borough', 'Neigborhood']
for row in table.find_all('tr'):
    col = row.find_all('td')
    fullrow = [row.text.strip() for row in col]
    content.append(fullrow)
df = pd.DataFrame(content[1:], columns=columns)
df.head(10)

,PostalCode,Borough,Neigborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


I need to clean the data as specified:

>* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
>* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
>* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [4]:
# Replace of Not assigned by nan values so I can better wrangle the data
dfclean = df.replace('Not assigned', np.nan).dropna(subset=['Borough']).copy()
# Copy of the value in Borough to nan values in corresponding Neigborhood
dfclean.Neigborhood.fillna(df.Borough, inplace=True)
dfclean.head(10)

,PostalCode,Borough,Neigborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [5]:
# Now I group by Postal Code and Borough joining the Neigborhood when there are more than 1 value per Postal Code
df_Toronto_grouped = dfclean.groupby(['PostalCode','Borough'])['Neigborhood'].apply(', '.join).reset_index().drop_duplicates()
df_Toronto_grouped.head(10)

,PostalCode,Borough,Neigborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Finally, the total number of Postal Codes retrieved are as follows:

In [6]:
df_Toronto_grouped.shape

(103, 3)

In [7]:
df_Toronto_grouped.to_csv(path_or_buf='PC_N_Toronto.csv')